Source: https://esa-worldcover.org/en/data-access (see AWS section). Wasn't able to find grid for 2021 so copied the grid from 2020 and made necessary adjustments.

Must have `tile-mate` installed in editable mode. Also, uses `dem_stitcher` for serialization (available via conda-forge)

In [1]:
import geopandas as gpd
from tile_mate.tile_model import TILE_SCHEMA
from tile_mate.stitcher import DATA_DIR
import pandera as pa
from dem_stitcher.geojson_io import to_geojson_gzip

In [2]:
s3_url_prefix = "https://esa-worldcover.s3.eu-central-1.amazonaws.com"

# load worldcover grid
url_2020 = f'{s3_url_prefix}/v100/2020/esa_worldcover_2020_grid.geojson'
df_wc_tiles_2020_src = gpd.read_file(url_2020)

In [3]:
def url_gen(tile_id):
    url = f"{s3_url_prefix}/v100/2020/map/ESA_WorldCover_10m_2020_v100_{tile_id}_Map.tif"
    return url

df_wc_tiles_2020 = df_wc_tiles_2020_src.copy()
df_wc_tiles_2020['url'] = df_wc_tiles_2020.ll_tile.map(url_gen)
df_wc_tiles_2020.rename(columns={'ll_tile': 'tile_id'}, inplace=True)
df_wc_tiles_2020 = df_wc_tiles_2020[['tile_id', 'url', 'geometry']]
df_wc_tiles_2020.head()

,tile_id,url,geometry
0,S54E168,https://esa-worldcover.s3.eu-central-1.amazona...,"POLYGON ((168.80786 -52.75980, 168.79761 -52.7..."
1,S54E165,https://esa-worldcover.s3.eu-central-1.amazona...,"POLYGON ((166.50177 -51.00000, 166.46563 -51.0..."
2,S57E159,https://esa-worldcover.s3.eu-central-1.amazona...,"POLYGON ((159.00000 -54.17474, 159.05881 -54.1..."
3,S57E156,https://esa-worldcover.s3.eu-central-1.amazona...,"POLYGON ((159.00000 -55.01738, 158.96296 -55.0..."
4,S45E144,https://esa-worldcover.s3.eu-central-1.amazona...,"POLYGON ((147.00000 -42.00000, 147.00000 -43.9..."


In [4]:
df_wc_tiles_2020 = TILE_SCHEMA.validate(df_wc_tiles_2020)

In [5]:
df_wc_tiles_2020.url[0]

'https://esa-worldcover.s3.eu-central-1.amazonaws.com/v100/2020/map/ESA_WorldCover_10m_2020_v100_S54E168_Map.tif'

In [6]:
df_wc_tiles_2021 = df_wc_tiles_2020.copy()
df_wc_tiles_2021['url'] = df_wc_tiles_2021.url.map(lambda url: url.replace('v100', 'v200').replace('2020', '2021'))
df_wc_tiles_2021.url[0]

'https://esa-worldcover.s3.eu-central-1.amazonaws.com/v200/2021/map/ESA_WorldCover_10m_2021_v200_S54E168_Map.tif'

# Serialize

In [37]:
to_geojson_gzip(df_wc_tiles_2021, str(DATA_DIR / 'esa_world_cover_2021.geojson.zip'))
to_geojson_gzip(df_wc_tiles_2020, str(DATA_DIR / 'esa_world_cover_2020.geojson.zip'))

'/Users/cmarshak/bekaert-team/tile-mate/tile_mate/data/esa_world_cover_2020.geojson.zip'